In [ ]:
import requests
import json
import pandas as pd


ModuleNotFoundError: No module named 'polars'

In [ ]:

# List of years
years = [2017, 2018, 2019, 2020, 2021, 2022, 2023]

# Initialize an empty list to store the extracted data
extracted_data = []
party_data = []

# Loop through each year
for year in years:
    # URL of the JSON data
    url = f'https://zpravy.udhpsh.cz/export/vfz{year}-index.json'

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON data
        data = response.json()

        # Extract relevant data
        for party in data['parties']:
            for file in party['files']:
                extracted_data.append({
                    'year': year,
                    'key': data['election']['key'],
                    'ic': party['ic'],
                    'subject': file['subject'],
                    'url': file['url']
                })
                party_data.append({
                    'ic': party['ic'],
                    'shortName': party['shortName'],
                    'longName': party['longName']

                })
                
    else:
        print(f"Failed to retrieve data for year {year}: {response.status_code}")

# Create DataFrame
df = pd.DataFrame(extracted_data)
df_party = pd.DataFrame(party_data).drop_duplicates(subset=['ic'])
df_party.rename(columns={ 'ic': 'party_key','shortName': 'party_short', 'longName': 'party_long'}, inplace=True)

df_party.to_csv('/Users/zackrawr/Developer/Python_WS/public_service/PBI/D_party.csv', index=False)
# Print the DataFrame
print(len(df))

In [ ]:
df_cprijmy = df[df['subject'] == 'cprijmy']
df_cvydaje = df[df['subject'] == 'cvydaje']
df_zamest = df[df['subject'] == 'zamest']
df_polinst = df[df['subject'] == 'polinst']
df_podil = df[df['subject'] == 'podil']
df_penizefo = df[df['subject'] == 'penizefo']
df_bupfo = df[df['subject'] == 'bupfo']
df_penizepo = df[df['subject'] == 'penizepo']
df_buppo = df[df['subject'] == 'buppo']
df_dluhy = df[df['subject'] == 'dluhy']
df_dedictvi = df[df['subject'] == 'dedictvi']
df_clenove = df[df['subject'] == 'clenove']


In [ ]:
subjects = [
df_cprijmy,
df_cvydaje,
df_zamest,
df_podil,
df_penizefo,
df_penizepo,
# df_polinst,
# df_bupfo,
# df_buppo,
# df_dluhy,
# df_dedictvi,
# df_clenove,
]

In [ ]:
def fetch_json(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        return response.json()  # Return parsed JSON
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        return None  # Return None in case of error

# Fetch JSON for each URL and store it in a new column
# df_head['json_response'] = df_head['url'].apply(fetch_json)
# df_head.to_csv('financovani.csv', index=False)

In [ ]:
for subject in subjects:
    subject['json_response'] = subject['url'].apply(fetch_json)
    subject.to_csv(f'{subject.iloc[0]["subject"]}.csv', index=False, encoding='utf-8')
    print(f'{subject.iloc[0]["subject"]} done')
    
    # Step 1: Expand the JSON into a tabular format
    exploded_df = subject.explode('json_response')

    # Step 2: Normalize the JSON data
    expanded_df = pd.json_normalize(exploded_df['json_response'])

    # Add meaningful context from the original DataFrame
    result_df = exploded_df[['year',  'ic']].reset_index(drop=True)
    result_df = pd.concat([result_df, expanded_df], axis=1)
    result_df.to_csv(f'/Users/zackrawr/Developer/Python_WS/public_service/CURATED/{subject.iloc[0]["subject"]}.csv', index=False, encoding='utf-8')
    print(f'{subject.iloc[0]["subject"]} done in curated')


### processing FO

In [3]:
df_fo = pd.read_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/penizefo.csv')
df_fo = df_fo.drop(columns=['year', 'ic', 'date', 'money', 'addrCity'])
df_fo = df_fo.drop_duplicates(subset=['lastName', 'firstName', 'birthDate'])
df_fo['donor_key'] =  'fo.' + df_fo.index.astype(str)
df_fo['donor_form'] =  'FO'
df_fo['titleBefore'] = df_fo['titleBefore'].fillna('')
df_fo['titleAfter'] = df_fo['titleAfter'].fillna('')
df_fo['donor_name'] = df_fo.apply(lambda row: f"{row['titleBefore']} {row['firstName']} {row['lastName']} {row['titleAfter']}".strip(), axis=1)
df_fo['donor_name'] = df_fo['donor_name'].str.replace('  ', ' ')  # Remove any double spaces
df_fo.head()
df_fo.to_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/D_FO.csv', index=False, encoding='utf-8')

In [ ]:
df_penizefo = pd.read_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/penizefo.csv')
df_fo = pd.read_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/D_FO.csv')
df_merged = pd.merge(df_penizefo, df_fo, on=['lastName', 'firstName', 'birthDate'], how='left')
result_df = df_merged.drop(['lastName', 'firstName', 'titleBefore_x', 'titleAfter_x', 'birthDate', 'addrCity','titleBefore_y','titleAfter_y','donor_name'], axis=1)
result_df.to_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/F_dary_fo.csv', index=False, encoding='utf-8')

### processing PO

In [4]:
df_po = pd.read_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/penizepo.csv')
df_po = df_po.drop(columns=['year', 'ic', 'date', 'money', 'addrStreet', 'addrCity', 'addrZip'])
df_po = df_po.drop_duplicates(subset=['companyId'])
df_po['donor_key'] =  'po.' + df_po.index.astype(str)
df_po['donor_form'] =  'PO'
df_po = df_po.rename(columns={'company': 'donor_name'})
df_po.head()
df_po.to_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/D_PO.csv', index=False, encoding='utf-8')

In [ ]:
df_penizepo = pd.read_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/penizepo.csv')
df_po = pd.read_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/D_PO.csv')
df_merged = pd.merge(df_penizepo, df_po, on=['companyId'], how='left')
result_df = df_merged.drop(['companyId', 'company','addrStreet', 'addrCity', 'addrZip', 'donor_name'], axis=1)
result_df.to_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/F_dary_po.csv', index=False, encoding='utf-8')

### processing dary

In [12]:
df_penizepo = pd.read_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/F_dary_po.csv')
df_penizefo = pd.read_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/F_dary_fo.csv')
df_merged = pd.concat([df_penizepo, df_penizefo])
df_merged['year'] =  df_merged['year'].astype(str) + '-01-01'
df_merged.rename(columns={'year': 'date_year', 'ic': 'party_key', 'date': 'donor_date', 'money': 'donor_amount'}, inplace=True)
df_merged = df_merged[df_merged['donor_amount'] != 0]
df_merged.to_csv('/Users/zackrawr/Developer/Python_WS/public_service/PBI/F_dary.csv', index=False, encoding='utf-8')

In [11]:
print(len(df_merged))
df_merged = df_merged[df_merged['donor_amount'] != 0]
print(len(df_merged))

128096
112642


### processing rozvaha

In [ ]:
cprijmy = pd.read_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/cprijmy.csv')
cvydaje = pd.read_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/cvydaje.csv')
cprijmy['key'] = 'P_' + cprijmy['key'].astype(str)
cvydaje['key'] = 'V_' + cvydaje['key'].astype(str)
df_merged = pd.concat([cprijmy, cvydaje])
df_merged = df_merged.drop(columns=['description'])
df_merged['year'] =  df_merged['year'].astype(str) + '-01-01'
df_merged.rename(columns={'year': 'date_year', 'ic': 'party_key', 'key': 'bod_key'}, inplace=True)
df_merged.to_csv('/Users/zackrawr/Developer/Python_WS/public_service/PBI/F_rozvaha.csv', index=False, encoding='utf-8')

### processing donors

In [9]:
df_po = pd.read_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/D_PO.csv')
df_fo = pd.read_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/D_FO.csv')
df_merged = pd.concat([df_po, df_fo])
df_merged = df_merged[['donor_key','donor_name','donor_form','birthDate','companyId']] 
df_merged.rename(columns={'birthDate': 'date_DOB', 'companyId': 'ICO'}, inplace=True)
df_merged['donor_name'] =df_merged['donor_name'].str.strip('"').str.replace('"', '')
df_merged.to_csv('/Users/zackrawr/Developer/Python_WS/public_service/PBI/D_donor.csv', index=False, encoding='utf-8')

In [8]:
df_stripped = df_merged
df_stripped['donor_name'] =df_stripped['donor_name'].str.strip('"').str.replace('"', '')
df_stripped.head()

,donor_key,donor_name,donor_form,date_DOB,ICO
0,po.0,"ABF, a.s.",PO,NaN,63080575
1,po.2,"Advokátní kancelář Belha, Vacíř & spol., s.r.o.",PO,NaN,28498038
2,po.3,DDX Czech a.s.,PO,NaN,5401275
3,po.4,ERDEX a.s.,PO,NaN,27182533
4,po.5,Finep Prosek bytová k.s.,PO,NaN,24716847


### processing zamestnanci

In [5]:
df_zamest = pd.read_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/zamest.csv')
df_zamest['year'] =  df_zamest['year'].astype(str) + '-01-01'
df_zamest.rename(columns={'year': 'date_year', 'ic': 'party_key', 'job': 'job_desc', 'number': 'job_count'}, inplace=True)

df_job = pd.read_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/D_job.csv')
df_job.reset_index(inplace=True, drop=False)
df_job.rename(columns={'index': 'job_key'}, inplace=True)
df_job.to_csv('/Users/zackrawr/Developer/Python_WS/public_service/PBI/D_job.csv', index=False, encoding='utf-8')

df_merged = pd.merge(df_zamest, df_job, on=['job_desc'], how='left')
df_merged = df_merged[['date_year','party_key','job_key','job_count']] 
df_merged.to_csv('/Users/zackrawr/Developer/Python_WS/public_service/PBI/F_zamestnanci.csv', index=False, encoding='utf-8')


In [3]:
df_job.head()

,job_key,job_desc,job_group
0,0,Administraivní práce,Administrative and Support Roles
1,1,administrativa,Administrative and Support Roles
2,2,Administrativa,Administrative and Support Roles
3,3,administrativa - DPČ - zam. malého rozsahu,Administrative and Support Roles
4,4,administrativa (05-09/2022),Administrative and Support Roles
